In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
import pymysql

In [2]:
connection = pymysql.connect(
        host='localhost',
        user='root',
        password='root',
        database='main_database'
    )

In [3]:
spark = SparkSession.builder.appName("hadoop_add_file").config("spark.jars", "C:\spark-3.5.1-bin-hadoop3\jars\mysql-connector-j-8.4.0.jar").getOrCreate()

In [4]:
url = "jdbc:mysql://localhost:3306/main_database"
properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.jdbc.Driver"
}

In [5]:
df = spark.read.jdbc(url=url, table='cf_etl_table', properties=properties)
df.show()
is_inc = df.select('is_incremental').collect()[0]['is_incremental']
is_inc

+---+--------------+-----------+--------------------+--------------+-------------------+-------------------+--------------+--------------+---------+
| id|  Schema_names|Table_names|            location|hdfs_file_name|    start_date_time|      end_date_time|is_incremental|execution_date|inc_field|
+---+--------------+-----------+--------------------+--------------+-------------------+-------------------+--------------+--------------+---------+
|  1|test_database1|sample_data|hdfs://localhost:...|   sample_data|2023-06-23 00:00:01|2023-06-23 23:59:59|             1|          NULL| Tnx_date|
+---+--------------+-----------+--------------------+--------------+-------------------+-------------------+--------------+--------------+---------+



1

In [6]:
df.show()

+---+--------------+-----------+--------------------+--------------+-------------------+-------------------+--------------+--------------+---------+
| id|  Schema_names|Table_names|            location|hdfs_file_name|    start_date_time|      end_date_time|is_incremental|execution_date|inc_field|
+---+--------------+-----------+--------------------+--------------+-------------------+-------------------+--------------+--------------+---------+
|  1|test_database1|sample_data|hdfs://localhost:...|   sample_data|2023-06-23 00:00:01|2023-06-23 23:59:59|             1|          NULL| Tnx_date|
+---+--------------+-----------+--------------------+--------------+-------------------+-------------------+--------------+--------------+---------+



In [16]:
def etl(url,table_name,properties):
    df = spark.read.jdbc(url=url, table=table_name, properties=properties)
    is_inc = df.select('is_incremental').collect()[0]['is_incremental']
    for index in range(df.count()):
        if is_inc:
            start_date = df.select('start_date_time').collect()[index]['start_date_time']
            end_date = df.select('end_date_time').collect()[index]['end_date_time']
            location = df.select('location').collect()[index]['location']
            hdfs_file_name = df.select('hdfs_file_name').collect()[index]['hdfs_file_name']
            inc_field = df.select('inc_field').collect()[index]['inc_field']
            database_name = df.select('Schema_names').collect()[index]['Schema_names']
            table_name = df.select('Table_names').collect()[index]['Table_names']
            hdfs_path = f'{location}{hdfs_file_name}'
            jdbc_url = f"jdbc:mysql://localhost:3306/{database_name}"
            query =  f"(SELECT * FROM {database_name}.{table_name} WHERE {inc_field} BETWEEN '{start_date}' AND '{end_date}') as selected_data"
            dataframe = spark.read.jdbc(url=jdbc_url, table=query, properties=properties)
            dataframe.write.mode('append').parquet(hdfs_path)
            dataframe.write.csv('./output_data/out_put.csv',mode = 'append')
            with connection.cursor() as cursor:
               exec_date = f'update `main_database`.cf_etl_table set execution_date = (current_timestamp) where id = {index +1}' 
               cursor.execute(exec_date)
               start_date = f'update `main_database`.cf_etl_table set start_date_time = date_add(start_date_time, interval 1 day)'
               cursor.execute(start_date)
               end_date = f'update `main_database`.cf_etl_table set end_date_time = date_add(end_date_time, interval 1 day)'
               cursor.execute(end_date)
               connection.commit()

In [17]:
etl(url,'cf_etl_table',properties)